In [ ]:
import os
import numpy as np
import geopandas as gpd
import pandas as pd
from joblib import Parallel, delayed
import shapely as sh

In [ ]:
chipsets_file = "/opt/earthdata/chipsets-definitions.parquet"
basedir = "/opt/earthdata/chipsets"
dataset = "s1grd-2022"

In [ ]:
cs = gpd.read_parquet(chipsets_file)
cs.shape

In [ ]:
from shapely import wkt
with open("../data/luxembourg.wkt") as f:
    lux = wkt.load(f)
levels = {
          'L0': lux,
          'L1': lux.buffer(1),
          'L2': lux.buffer(2),
          'L3': lux.buffer(3),
          'L4': lux.buffer(4),
    
         }

In [ ]:
for k,v in levels.items():
    with open(f"../data/luxembourg_{k}.wkt", "w") as f:
        wkt.dump(v, f)

In [ ]:
all_levels = sh.ops.unary_union(list(levels.values()))
luxc = cs[[gi.intersects(all_levels) for gi in cs.geometry]]
luxc

In [ ]:
def chiplevel(geom, levels):
    for l in sorted(list(levels.keys())):
        if levels[l].intersects(geom):
            return l
    return None

for chipset_id in luxc.index:
    
    print (f"adding levels to {chipset_id}")
    
    # look for defs file
    chipset_defs_file = [f for f in os.listdir(basedir) if chipset_id in f and f.endswith('.geojson')]
    if len(chipset_defs_file)==0:
        raise ValueError(f"could not file defs file for {chipset_id}")
    elif len(chipset_defs_file)>1:
        raise ValueError(f"too many defs files for {chipset_id}")

    chipset_defs_file = chipset_defs_file[0]
    chipset_defs_file = f"{basedir}/{chipset_defs_file}"
    c = gpd.read_file(chipset_defs_file)

    
    # add level groups if they havent been added already
    c['level'] = [chiplevel(gi, levels) for gi in c.geometry.values]
    newgroups = []
    for _, row in c.iterrows():
        lgroups = row.group.split(",")
        if row.level is not None and not row.level in lgroups:
            g = ",".join(lgroups + [row.level])  
        else:
            g = row.group
        newgroups.append(g)
    c['group'] = newgroups
    del(c['level'])
    c.to_file(chipset_defs_file, driver='GeoJSON')

In [ ]:
chipset_id = luxc.index[2]
chipset_defs_file = [f for f in os.listdir(basedir) if chipset_id in f and f.endswith('.geojson')]
if len(chipset_defs_file)==0:
    raise ValueError(f"could not file defs file for {chipset_id}")
elif len(chipset_defs_file)>1:
    raise ValueError(f"too many defs files for {chipset_id}")

chipset_defs_file = chipset_defs_file[0]
chipset_defs_file = f"{basedir}/{chipset_defs_file}"
c = gpd.read_file(chipset_defs_file)


In [ ]:
c['color'] = [ gi.split(",")[1] if len(gi.split(","))>1 else None for gi in c['group'].values]

In [ ]:
c.explore(column='color')